In [ ]:
import numpy as np
import pandas as pd
import math


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/neuronske/data/full_sequences.pkl')
data.head()

training = data[data['event_label'] == 'Blender']
X = np.array(training.mfccs.tolist())
y = np.array(training.combine_squence_per_filename_and_event.tolist())

X = np.einsum('ijk->ikj', X)

In [ ]:
# Optional -> use only when testing gradients (holds changes in MFCC values instead of actual MFCCs)
def distance(a, b): return (max(a, b) - min(a, b)) * (-1 if a > b else 1) 

New_X = []

for x_instance in X:
  new_arr = []
  for i in range(0, x_instance.shape[0] - 1):
    gradient_arr = []
    for dim in range(0, x_instance.shape[1]):
      gradient_arr.append(distance(x_instance[i][dim], x_instance[i+1][dim]))
    new_arr.append(gradient_arr)
  new_arr.append(new_arr[429])
  New_X.append(new_arr)

In [ ]:
X = np.asarray(New_X)

In [ ]:
X = np.divide(New_X, np.amax(New_X))

In [ ]:
print(np.asarray(New_X).shape)
print(X.shape)

(392, 431, 13)
(319, 431, 13)


In [ ]:
indices = np.arange(X.shape[0])

# split the dataset 
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test, I_train, I_test = train_test_split(X, y, indices, test_size=0.2, random_state = 42)

In [ ]:
def create_output_seq(y_orig):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    is_one = False
    new_sequence_beginning = []
    new_sequence_end = []
    to_assign_beginning = 0
    to_assign_end = 0
    for label in sequence:
      if label == 1:
        if is_one:
          to_assign_beginning = 0
        else:
          to_assign_beginning = 1
        is_one = True
      else:
        if is_one:
          new_sequence_end[len(new_sequence_end)-1] = 1
          to_assign_end = 0
        else:
          to_assign_end = 0
        is_one = False
      new_sequence_beginning.append(to_assign_beginning)
      new_sequence_end.append(to_assign_end)
    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
# Optimizovana sa paddingom (prosirenjem indikatora pocetka i kraja) - umesto jednog broja za pocetak i kraj -> (expansion_num * 2 +- 1) broj 
def create_output_seq_expanded(y_orig, expansion_num):
  result_beginning = []
  result_end = []
  for sequence in y_orig:
    new_sequence_beginning = [0] * len(sequence)
    new_sequence_end = [0] * len(sequence)
    indexes_beginning = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 0 and sequence[i+1] == 1] 
    indexes_end = [i for i in range(len(sequence)) if sequence[i] == 1 and i-1>=0 and i+1 < len(sequence) and sequence[i-1] == 1 and sequence[i+1] == 0] 

    if sequence[len(sequence)-1] == 1:
      indexes_end.append(len(sequence))
    for index in indexes_beginning:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_beginning[start:end] = [1] * (end - start)

    for index in indexes_end:
      start = index - expansion_num if index - expansion_num >= 0 else index
      end = index + expansion_num if index + expansion_num < len(sequence) else index
      new_sequence_end[start:end] = [1] * (end - start)

    result_beginning.append(new_sequence_beginning)
    result_end.append(new_sequence_end)    
  return np.asarray(result_beginning), np.asarray(result_end)

In [ ]:
y_new_beginning_exp, y_new_end_exp = create_output_seq_expanded(y_train, 2)
ind = 34
print(y_train[ind])
print(80*'-')
print(y_new_beginning_exp[ind])
print(80*'-')
print(y_new_end_exp[ind])

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
--------------------------------------------------------------------------------
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [ ]:
y_new_beginning, y_new_end = create_output_seq(y_train)

In [ ]:
y_new_beginning[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
y_new_end[1]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def chunks(lst, n):
  """Yield successive n-sized chunks from lst."""
  return ([lst[i:i + n] for i in range(0, len(lst), n)])


def split_sequences_into_seconds(X, y, seconds = 1):
  sequence_len = X.shape[1]
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)

  X_new = []
  y_new = []
  for i, sequence in enumerate(X):
    X_splits = chunks(sequence, ticks_per_chosen_second)
    y_splits = chunks(y[i], ticks_per_chosen_second)
    x_false_instances = np.array([]).reshape(0,X_splits[0].shape[1])
    for j, X_split in enumerate(X_splits):
      X_temp = X_split
      y_temp = y_splits[j]
      if len(x_false_instances) < ticks_per_chosen_second:
        false_indices = np.where(~y_temp)
        x_false_instances = np.append(x_false_instances, X_split[false_indices], axis=0)

      diff = ticks_per_chosen_second - len(X_temp)
      if diff != 0:
        y_temp = np.append(y_temp, [0] * diff)
        X_temp = np.append(X_temp, x_false_instances[0: diff], axis=0)
        
      X_new.append(X_temp)
      y_new.append(y_temp)


  return np.asarray(X_new), np.asarray(y_new)

X_train_splitted, y_new_beginning_exp_splitted = split_sequences_into_seconds(X_train, y_new_beginning_exp, 5)
X_train_splitted, y_new_end_exp_splitted = split_sequences_into_seconds(X_train, y_new_end_exp, 5)


In [ ]:
print(X_train.shape)
print(X_train_splitted.shape)


(348, 431, 13)
(696, 216, 13)


In [ ]:
input_seq_len = 431
output_shape = 431
dense_layer_size = 150
num_of_features = 13
# output_len = len(data)

batch_size = 128  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 128  # Latent dimensionality of the encoding space.

In [ ]:
# Define architecture of autoencoder model. 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import regularizers

def create_encoder_decoder(input_seq_len = 431, num_of_features = 13):
  # Define an input sequence and process it.

  encoder_inputs = keras.Input(shape=(input_seq_len, num_of_features))
  encoder = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True, dropout=0.2)
  encoder_outputs, state_h = encoder(encoder_inputs)

  encoder_states = [state_h]
  # repeat_vector = keras.layers.RepeatVector(input_seq_len)(encoder_outputs)
  # Set up the decoder, using `encoder_states` as initial state.
  # decoder_inputs = keras.Input(shape=(25, 773))

  # We set up our decoder to return full output sequences.
  # decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True, return_state=True)
  # decoder_outputs, _ = decoder_gru(encoder_outputs, initial_state=encoder_states)
  # decoder_dense = keras.layers.Dense(1, activation="sigmoid")
  # decoder_outputs = decoder_dense(decoder_outputs)

  decoder_beg = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_beg_outputs_0 = decoder_beg(encoder_outputs)

  # decoder_beg_1 = keras.layers.Dense(dense_layer_size, activation='relu')
  # decoder_beg_outputs_1 = decoder_beg_1(decoder_beg_outputs_0)

  decoder_beg_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_beg_outputs_2 = decoder_beg_2(decoder_beg_outputs_0)

  decoder_end = keras.layers.Dense(dense_layer_size, activation='relu')
  decoder_end_outputs_0 = decoder_end(encoder_outputs)

  # decoder_end_1 = keras.layers.Dense(dense_layer_size, activation='relu')
  # decoder_end_outputs_1 = decoder_end_1(decoder_end_outputs_0)

  decoder_end_2 = keras.layers.Dense(1, activation='sigmoid')
  decoder_end_outputs_2 = decoder_end_2(decoder_end_outputs_0)

  return encoder_inputs, decoder_beg_outputs_2, decoder_end_outputs_2
  


In [ ]:
encoder_inputs, decoder_outputs_beg, decoder_outputs_end = create_encoder_decoder(X_train_splitted.shape[1])
model = keras.Model(encoder_inputs, [decoder_outputs_beg, decoder_outputs_end])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 216, 13)]    0                                            
__________________________________________________________________________________________________
gru_1 (GRU)                     [(None, 216, 128), ( 54912       input_2[0][0]                    
__________________________________________________________________________________________________
dense_4 (Dense)                 (None, 216, 150)     19350       gru_1[0][0]                      
__________________________________________________________________________________________________
dense_6 (Dense)                 (None, 216, 150)     19350       gru_1[0][0]                      
____________________________________________________________________________________________

In [ ]:
from keras.metrics import BinaryAccuracy, Precision, Recall, AUC

METRICS = [
      Precision(name='precision'),
      Recall(name='recall'),
      AUC(name='auc'),
      AUC(name='prc', curve='PR'), # precision-recall curve
]

model.compile(
    optimizer="adam", loss='binary_crossentropy', metrics=METRICS
)
model.fit(
    [X_train_splitted],
    [y_new_beginning_exp_splitted, y_new_end_exp_splitted],
    batch_size=batch_size,
    epochs=700,
    validation_split=0.2,
)

Epoch 1/700
5/5 [==============================] - 16s 454ms/step - loss: 1.4127 - dense_5_loss: 0.7190 - dense_7_loss: 0.6937 - dense_5_precision: 0.0116 - dense_5_recall: 0.7165 - dense_5_auc: 0.5192 - dense_5_prc: 0.0122 - dense_7_precision: 0.0113 - dense_7_recall: 0.6782 - dense_7_auc: 0.5256 - dense_7_prc: 0.0118 - val_loss: 0.3149 - val_dense_5_loss: 0.1513 - val_dense_7_loss: 0.1635 - val_dense_5_precision: 0.0113 - val_dense_5_recall: 0.3879 - val_dense_5_auc: 0.5182 - val_dense_5_prc: 0.0114 - val_dense_7_precision: 0.0113 - val_dense_7_recall: 0.3699 - val_dense_7_auc: 0.5211 - val_dense_7_prc: 0.0113
Epoch 2/700
5/5 [==============================] - 0s 40ms/step - loss: 0.1790 - dense_5_loss: 0.0989 - dense_7_loss: 0.0801 - dense_5_precision: 0.0113 - dense_5_recall: 0.2577 - dense_5_auc: 0.5077 - dense_5_prc: 0.0111 - dense_7_precision: 0.0113 - dense_7_recall: 0.2443 - dense_7_auc: 0.5094 - dense_7_prc: 0.0111 - val_loss: 0.1233 - val_dense_5_loss: 0.0663 - val_dense_7_l

In [ ]:
# dense_9_precision: 0.7723 - dense_9_recall: 0.5137

In [ ]:
# CONCLUSIONS: TOO MUCH FALSE NEGATIVES - BAD RECALL
#              BIGGER LATENT_TIM -> BETTER RESULTS -- LAT_DIM-F1 : 13-68, 64-70 256-73
def simple_evaluation(y_pred, y_test):
  tp = 0
  tn = 0
  fp = 0
  fn = 0

  for i, pred_arr in enumerate(y_pred):
    for j, pred_value in enumerate(pred_arr):
      predicted = pred_value
      real = y_test[i][j]
      if predicted and real:
        tp = tp + 1
      elif not(predicted) and real:
        fn = fn + 1
      elif predicted and not(real):
        fp = fp + 1
      elif not(predicted) and not(real):
        tn = tn + 1
    # break

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)

  print("PRECISION: " + str(precision))
  print("RECALL: " + str(recall))
  print("F1: " + str(f1))
  print("-"*20)
  print("tp: " + str(tp))
  print("tn: " + str(tn))
  print("fp: " + str(fp))
  print("fn: " + str(fn))



In [ ]:
X_test_splitted, y_test_splitted = split_sequences_into_seconds(X_test, y_test, 5)



In [ ]:
predicted_beg, predicted_end = model.predict(X_test_splitted)


In [ ]:
predicted_beg_rounded = [np.round(np.concatenate(i, axis=0), 0).astype(int) for i in predicted_beg]
predicted_end_rounded = [np.round(np.concatenate(i, axis=0), 0).astype(int) for i in predicted_end]


In [ ]:
simple_evaluation(predicted_beg_rounded,y_new_beginning_exp)
simple_evaluation(predicted_end_rounded,y_new_end_exp)


ZeroDivisionError: ignored

In [ ]:
def combine_beginning_end_exp(y_beg, y_end):
  result_begining = []
  result_ending = []

  for j in range(0, len(y_beg)):
    y_beg_curr = np.asarray(y_beg[j])
    y_end_curr = np.asarray(y_end[j])
    y_final = []

    y_beg_indices = np.where(y_beg_curr == 1)[0]
    y_end_indices = np.where(y_end_curr == 1)[0]


    y_beg_groups = []
    group = []
    for i, value in enumerate(y_beg_indices):
      if i == 0:
        group.append(value)
        continue
      if (value - y_beg_indices[i-1]) > 2 or i == len(y_beg_indices) - 1:
        if i == len(y_beg_indices) - 1:
          group.append(value)
        y_beg_groups.append(group.copy())
        group = []
        group.append(value)
      else:
        group.append(value)

    y_end_groups = []
    group = []
    for i, value in enumerate(y_end_indices):
      if i == 0:
        group.append(value)
        continue
      if (value - y_end_indices[i-1]) > 2 or i == len(y_end_indices) - 1:
        if i == len(y_end_indices) - 1:
          group.append(value)
        y_end_groups.append(group.copy())
        group = []
        group.append(value)
      else:
        group.append(value)


    new_sequence_beginning = [0] * len(y_beg_curr)
    new_sequence_ending = [0] * len(y_end_curr)

    for group in y_beg_groups:
      start = group[0]
      end = group[len(group)-1]
      new_sequence_beginning[math.floor((start + end) / 2)] = 1

    for group in y_end_groups:
      start = group[0]
      end = group[len(group)-1]
      new_sequence_ending[math.ceil((start + end) / 2)] = 1

    result_begining.append(new_sequence_beginning)
    result_ending.append(new_sequence_ending)

  return combine_beginning_end(result_begining, result_ending)


In [ ]:
print(np.asarray(predicted_end_rounded).shape)

(176, 216)


In [ ]:
def combine_predicted_to_original(y_pred, seconds, sequence_len):
  ticks_per_chosen_second = math.ceil(sequence_len / (10 / seconds))
  num_of_parts_of_sequence = math.ceil(sequence_len / ticks_per_chosen_second)
  y_pred_combined = []
  # Num of arrays at the end
  final_size = math.floor(y_pred.shape[0] / num_of_parts_of_sequence)

  print(num_of_parts_of_sequence)
  print(final_size)

  for i in range(0, final_size):
    current_arr = np.array([]).reshape(0,y_pred[0].shape[1])
    for j in range(0, num_of_parts_of_sequence):
      current_arr = np.concatenate((current_arr, y_pred[i * num_of_parts_of_sequence + j])).astype(int)
    y_pred_combined.append(current_arr)
  return y_pred_combined
predicted_end_rounded = np.expand_dims(predicted_end_rounded, axis=-1)
predicted_beg_rounded = np.expand_dims(predicted_beg_rounded, axis=-1)
  
predicted_end_rounded = combine_predicted_to_original(np.asarray(predicted_end_rounded), 5, 431)
predicted_beg_rounded = combine_predicted_to_original(np.asarray(predicted_beg_rounded), 5, 431)

2
88
2
88


In [ ]:
res = combine_beginning_end_exp(predicted_beg_rounded, predicted_end_rounded)

In [ ]:
print(np.asarray(res).shape)
print(res[1])

(88, 432)
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
def combine_beginning_end(y_beg, y_end):
  result = []
  
  for j in range(0, len(y_beg)):
    y_beg_curr = y_beg[j]
    y_end_curr = y_end[j]
    y_final = []
    was_one = False
    to_append_new = 0
    to_append = 0
    for i in range(0, len(y_beg_curr)):
      if was_one and i == len(y_beg_curr) - 1:
        average_sound_duration = int((i - index_last_beginning) / 2)
        y_final[index_last_beginning + average_sound_duration:i] = [0] * (i - index_last_beginning - average_sound_duration)
        to_append_new = 0
      if y_beg_curr[i] == 1:
        if was_one:
          average_sound_duration = int((i - index_last_beginning) / 2)
          y_final[index_last_beginning + average_sound_duration:i] = [0] * (i - index_last_beginning - average_sound_duration)
        else:
          to_append_new = 1
          was_one = True
        index_last_beginning = i  
        
      if y_end_curr[i] == 1:
        if was_one:
          to_append_new = 0
          was_one = False
      y_final.append(to_append) 
      to_append = to_append_new 
    result.append(y_final)  
  return result 


In [ ]:
y_test_splitted = np.expand_dims(y_test_splitted, axis=-1)
combined_test_y = combine_predicted_to_original(y_test_splitted, 5, 431)
simple_evaluation(res,combined_test_y)

In [ ]:
# Function defined later :-)
references, estimations = prepare_predicted_sequences_for_evaluation_and_evaluate('Blender', res, I_test, training, events, 1)

Accuracy: 0.5727155727155727
Segment based metrics
  Evaluated length                  : 758.19 sec
  Evaluated files                   : 88 
  Segment length                    : 1.00 sec

  Overall metrics (micro-average)
  F-measure
    F-measure (F1)                  : 26.22 %
    Precision                       : 71.08 %
    Recall                          : 16.08 %
  Error rate
    Error rate (ER)                 : 0.90 
    Substitution rate               : 0.00 
    Deletion rate                   : 0.84 
    Insertion rate                  : 0.07 
  Accuracy
    Sensitivity                     : 16.08 %
    Specificity                     : 94.15 %
    Balanced accuracy               : 55.11 %
    Accuracy                        : 57.27 %

  Class-wise average metrics (macro-average)
  F-measure
    F-measure (F1)                  : 26.22 %
    Precision                       : 71.08 %
    Recall                          : 16.08 %
  Error rate
    Error rate (ER)              

In [ ]:
!pip install sed_eval

     |████████████████████████████████| 2.1MB 15.3MB/s 
  Created wheel for sed-eval: filename=sed_eval-0.2.1-cp37-none-any.whl size=26124 sha256=23095859a7eba7479805044e1e24599337067143cc2fb86cc6d2eba3633154e9
  Stored in directory: /root/.cache/pip/wheels/3c/72/5d/5fc941f98c583ce9d8adee7f13e24b46459aeded4125f9c369
  Created wheel for dcase-util: filename=dcase_util-0.2.18-cp37-none-any.whl size=2147233 sha256=8b4a5ba0305a8e77880628da0c8c75fcfe01d07bab17c10017af8c10b4c0f2da
  Stored in directory: /root/.cache/pip/wheels/d1/5d/9e/c94cd8bfb80541c73d549c734cafcd3de994dbb117a6485c81
Successfully built sed-eval dcase-util


In [ ]:
events = pd.read_csv("/content/drive/MyDrive/neuronske/data/Synthetic_dataset/synthetic_dataset.csv", delimiter="\t") 
events.head()

,filename,onset,offset,event_label
0,10.wav,0.287159,1.057712,Alarm_bell_ringing
1,10.wav,1.717124,1.967124,Alarm_bell_ringing
2,10.wav,5.049573,6.013632,Alarm_bell_ringing
3,10.wav,7.240901,8.492352,Speech
4,1000.wav,4.352958,4.664092,Alarm_bell_ringing


In [ ]:
import sed_eval
import dcase_util

def return_onsets_and_offsets_for_sequence(sequence, duration_seconds=10):
  single_tick_in_seconds = duration_seconds / len(sequence)
  onsets = []
  offsets = []

  error_allowance = 5
  error_counter = 0
  in_positive_sequence = 0
  
  for index, tick in enumerate(sequence):
    if tick and not(in_positive_sequence):
      in_positive_sequence = 1
      onsets.append(index * single_tick_in_seconds)
    elif not(tick) and in_positive_sequence:
      if error_counter >= error_allowance:
        error_counter = 0
        in_positive_sequence = 0
        offsets.append((index - error_allowance) * single_tick_in_seconds)
      else:
        error_counter = error_counter + 1
    elif tick and in_positive_sequence and error_counter:
      error_counter = 0
  if in_positive_sequence:
    offsets.append((index) * single_tick_in_seconds)

  return onsets, offsets

def prepare_predicted_sequences_for_evaluation_and_evaluate(event_label, y_pred, I_test, training_data, events_data, return_prepared_data_flag = 0):
  references = []
  estimations = []

  for i, y_predicted in enumerate(y_pred):
    file_index = I_test[i]
    file_name = training_data.iloc[file_index]['filename']

    onsets, offsets = return_onsets_and_offsets_for_sequence(y_predicted, 10)

    for j, onset in enumerate(onsets):
      estimations.append({
          'event_label': event_label,
          'event_onset': onset,
          'event_offset': offsets[j],
          'file': file_name
      })

    reference_events = events_data[(events_data['filename'] == file_name) & (events_data['event_label'] == event_label)]
    for ref_event in np.asarray(reference_events):
      references.append({
          'event_label': event_label,
          'event_onset': ref_event[1],
          'event_offset': ref_event[2],
          'file': file_name
      })

  evaluate_and_print_results(references, estimations)

  if return_prepared_data_flag:
    return references, estimations


def evaluate_and_print_results(references, estimations):
  reference_event_list = dcase_util.containers.MetaDataContainer(
      references
  )

  estimated_event_list = dcase_util.containers.MetaDataContainer(
      estimations
  )

  segment_based_metrics = sed_eval.sound_event.SegmentBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      time_resolution=1.0
  )
  event_based_metrics = sed_eval.sound_event.EventBasedMetrics(
      event_label_list=reference_event_list.unique_event_labels,
      t_collar=0.250,
      evaluate_onset=True,
      evaluate_offset=True,
      percentage_of_length=0.5
  )

  for filename in reference_event_list.unique_files:
      reference_event_list_for_current_file = reference_event_list.filter(
          filename=filename
      )

      estimated_event_list_for_current_file = estimated_event_list.filter(
          filename=filename
      )

      segment_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

      event_based_metrics.evaluate(
          reference_event_list=reference_event_list_for_current_file,
          estimated_event_list=estimated_event_list_for_current_file
      )

  # Get only certain metrics
  overall_segment_based_metrics = segment_based_metrics.results_overall_metrics()
  print("Accuracy:", overall_segment_based_metrics['accuracy']['accuracy'])

  # Or print all metrics as reports
  print(segment_based_metrics)
  print(event_based_metrics)